In [1]:
import pandas as pd
import os

# from sqlalchemy import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import * 

from env import user, password, host

# shhhhh..
import warnings
warnings.filterwarnings("ignore")

In [2]:
#create enviroment
spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/21 14:56:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/10/21 14:56:11 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
def get_db_url(db):
    return (f'mysql+pymysql://{user}:{password}@{host}/{db}')

#### This exercises use the cases, dept, and source tables from the 311_data on the Codeup MySQL server.

- Read the case, department, and source data into their own spark dataframes.

In [6]:
cases_query = '''SELECT * FROM cases'''
dept_query = '''SELECT * FROM dept'''
source_query = '''SELECT * FROM source'''
url = get_db_url("311_data")

In [5]:
cases_df = pd.read_sql(cases_query, url)

In [7]:
dept_df = pd.read_sql(dept_query, url)

In [8]:
source_df = pd.read_sql(source_query, url)

In [9]:
cases_df = spark.createDataFrame(cases_df)
cases_df.show(4, vertical=True)

22/10/21 15:08:57 WARN TaskSetManager: Stage 0 contains a task of very large size (12594 KiB). The maximum recommended task size is 1000 KiB.


22/10/21 15:09:02 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 0 (TID 0): Attempting to kill Python Worker
-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 SLA_due_date         | 9/26/20 0:42         
 case_late            | NO                   
 num_days_late        | -998.5087616         
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
-RECORD 1------------------------------------
 case_id              | 1014127333           
 case_opened_date     | 1/1/18 0:46          
 case_closed_date     | 1/3/18 8:11   

In [12]:
dept_df = spark.createDataFrame(dept_df)
dept_df.show(4)

+--------------------+--------------------+----------------------+-------------------+
|       dept_division|           dept_name|standardized_dept_name|dept_subject_to_SLA|
+--------------------+--------------------+----------------------+-------------------+
|     311 Call Center|    Customer Service|      Customer Service|                YES|
|               Brush|Solid Waste Manag...|           Solid Waste|                YES|
|     Clean and Green|Parks and Recreation|    Parks & Recreation|                YES|
|Clean and Green N...|Parks and Recreation|    Parks & Recreation|                YES|
+--------------------+--------------------+----------------------+-------------------+
only showing top 4 rows



In [13]:
source_df = spark.createDataFrame(source_df)
source_df.show(4)

+-----+---------+----------------+
|index|source_id| source_username|
+-----+---------+----------------+
|    0|   100137|Merlene Blodgett|
|    1|   103582|     Carmen Cura|
|    2|   106463| Richard Sanchez|
|    3|   119403|  Betty De Hoyos|
+-----+---------+----------------+
only showing top 4 rows



Let's see how writing to the local disk works in spark:

- Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json


In [16]:
source_df.write.json('source.json')
source_df.write.csv('source.csv', header=True)

In [17]:
spark.read.json('source.json').show(5)

+-----+---------+-----------------+
|index|source_id|  source_username|
+-----+---------+-----------------+
|  121|  rb05270|       Rene Bomar|
|  122|  rs16746|Ronald S. Sorrell|
|  123|  ru26699|      Raul Ubides|
|  124|  sg22264|  Steven Martinez|
|  125|  sg26196|   Samantha Garza|
+-----+---------+-----------------+
only showing top 5 rows



In [18]:
spark.read.csv('source.csv', header=True).show(5)

+-----+---------+-----------------+
|index|source_id|  source_username|
+-----+---------+-----------------+
|  121|  rb05270|       Rene Bomar|
|  122|  rs16746|Ronald S. Sorrell|
|  123|  ru26699|      Raul Ubides|
|  124|  sg22264|  Steven Martinez|
|  125|  sg26196|   Samantha Garza|
+-----+---------+-----------------+
only showing top 5 rows



In [14]:
cases_df.write.csv('cases.csv', header=True)
spark.read.csv('cases.csv', header=True).show()

22/10/21 15:19:34 WARN TaskSetManager: Stage 6 contains a task of very large size (12594 KiB). The maximum recommended task size is 1000 KiB.


+----------+----------------+----------------+-------------+---------+-------------------+-----------+--------------------+--------------------+--------+-----------+------------+--------------------+----------------+
|   case_id|case_opened_date|case_closed_date| SLA_due_date|case_late|      num_days_late|case_closed|       dept_division|service_request_type|SLA_days|case_status|   source_id|     request_address|council_district|
+----------+----------------+----------------+-------------+---------+-------------------+-----------+--------------------+--------------------+--------+-----------+------------+--------------------+----------------+
|1013997231|    11/2/17 8:59|   11/6/17 14:50| 11/9/17 8:59|       NO|       -2.756319444|        YES|    Waste Collection|       Cart Delivery|     7.0|     Closed|      140987|8443  CORDOVA POI...|               4|
|1013997232|    11/2/17 8:59|   3/16/18 11:39| 3/22/18 8:59|       NO|       -5.889421296|        YES|Traffic Engineeri...|Traffic I

- Inspect your folder structure. What do you notice?


Forgot to use this format, 

>(
>    
>    source_df.write.format('csv')
>
>    .mode('overwrite')
>
>    .option('header', 'True')
>
>    .save('data/source_csv')
>
>)

so I don't know 'where' my files saved path wise.
> But looking at them in their directory at the side of VSCode, they're broken into different parts, which helps run quicker with less processing power.

- Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.